If you want to start experimenting with continuous-time temporal graph models, you’ll find that many of them are already available on GitHub. In fact, the ecosystem for continuous-time methods is currently stronger than for discrete-time ones: public libraries usually provide better benchmark datasets, training routines, and evaluation settings. This means you can get up and running more quickly, and it’s easier to compare your results with existing work.

# Reproduce results on TGB

TGB is a collection of challenging and diverse benchmark datasets for realistic, reproducible, and robust evaluation for machine learning on temporal graphs. TGB includes both dynamic link and node property prediction tasks and an automated pipeline from dataset downloading, dataloading, evaluation and submission to the TGB leaderboard.

**Reproducing results on TGB** is very easy and can be achieved using this repository https://github.com/fpour/TGB_Baselines

# Test your own model on TGB

Example adapted from https://github.com/manuel-dileo/lp-heuristics

Let's test a preferential attachment heuristic on a TGB dataset.


In [1]:
!pip install --upgrade --force-reinstall py-tgb


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 129.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 185.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 167.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [23]:
import torch
from torch_geometric.data import DataLoader
from torch_geometric.utils import degree
import numpy as np
from tqdm import tqdm
import numpy as np

def PA(A, edge_index, batch_size=100000):
    # The Preferential-Attachment heuristic score.
    # s(u,v) = deg(u) * deg(v)
    degree_col = A.sum(axis=0)
    degree_row = A.sum(axis=1)
    A_ = A.multiply(degree_col).multiply(degree_row).tocsr()

    link_loader = DataLoader(range(edge_index.size(1)), batch_size)
    scores = []

    for ind in link_loader:
        src = edge_index[0, ind].cpu().numpy().astype(np.int64)
        dst = edge_index[1, ind].cpu().numpy().astype(np.int64)

        cur_scores = np.array([A_[s, d] for s, d in zip(src, dst)])
        scores.append(cur_scores)

    scores = np.concatenate(scores, 0)
    return scores, edge_index



In [26]:
from tgb.linkproppred.dataset_pyg import PyGLinkPropPredDataset
from tgb.linkproppred.evaluate import Evaluator
from tgb.linkproppred.dataset_pyg import PyGLinkPropPredDataset

dataset_name = "tgbl-wiki"

dataset = PyGLinkPropPredDataset(name=dataset_name, root="datasets")
dataset.load_test_ns() #inductive negative samples
dataset.load_val_ns() #inductive negative samples
    
evaluator = Evaluator(name=dataset_name)

raw file found, skipping download
Dataset directory is  /root/venv/lib/python3.11/site-packages/tgb/datasets/tgbl_wiki
loading processed file


In [4]:
!pip install torch_geometric

# Optional dependencies:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.8.0+cpu.html


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Looking in links: https://data.pyg.org/whl/torch-2.8.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.2/647.2 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.4/751.4 kB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 28.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [29]:
from torch_geometric.data import Data
import torch
import numpy as np
import scipy.sparse as ssp
from tqdm import tqdm

# Number of nodes in the temporal graph
num_nodes = dataset.get_TemporalData().num_nodes

# --- Build the training graph ---
# Stack source and destination nodes from the training edges
#edge_index is the adjaceny matrix in format [[src_edges], [dst_edges]]
#Dataset has no node features, so specifying number nodes will create a node feature matrix of ones
#Otherwise, you can specifiy x as node feature matrix, e.g. x = torch.ones((num_nodes,1))
#This is a PyG Data object
data = Data(
    edge_index=torch.stack((dataset.src[dataset.train_mask], 
                            dataset.dst[dataset.train_mask])),
    num_nodes=num_nodes
)

# --- Convert training edges to sparse adjacency matrix ---
# This is efficient for computing pa scores
edge_weight = torch.ones(data.edge_index.size(1), dtype=int)  # all edges weight=1
A = ssp.csr_matrix(
    (edge_weight, (data.edge_index[0], data.edge_index[1])),
    shape=(num_nodes, num_nodes)
)

# Mask to select test edges
mask = dataset.test_mask

# --- Sample negative edges for evaluation ---
# For each positive test edge, generate non-existent edges (negatives)
# It will create a dict of list, each list contains negative edges for each positive edge.
neg_edges = dataset.negative_sampler.query_batch(
    dataset.src[mask],
    dataset.dst[mask],
    dataset.ts[mask], 
    split_mode='test'
)

# Metric used for evaluation (usually "mrr") [for format required by evaluator object]
metric = dataset.eval_metric

# Stack positive test edges for easy indexing
pos_edge = torch.stack((dataset.src[mask], dataset.dst[mask]))

# List to store per-edge performance
perf_list = []

# --- Main loop: compute PA scores and MRR for each test edge ---
for idx in tqdm(range(len(neg_edges))):
    #Recall that pos_edge is 2D tensor [[src_edges], [dst_edges]]
    neg_batch = neg_edges[idx]             # negative edges for this positive edge
    pos_src = pos_edge[0, idx]             # source node of positive edge
    pos_dst = pos_edge[1, idx]             # destination node of positive edge

    # Prepare query edges: first positive, then negatives
    query_src = torch.Tensor([int(pos_src) for _ in range(len(neg_batch) + 1)])
    query_dst = torch.Tensor([int(pos_dst)] + neg_batch)
    edge_index = torch.stack((query_src, query_dst))

    # Compute Preferential Attachment scores for the batch
    # y_pred[0] -> positive edge, y_pred[1:] -> negative edges
    y_pred, _ = PA(A, edge_index) #your own model

    # --- Compute MRR for this edge ---
    input_dict = {
        "y_pred_pos": np.array([y_pred[0]]),    # score for positive edge
        "y_pred_neg": np.array(y_pred[1:]),     # scores for negatives
        "eval_metric": [metric]                 # evaluation metric
    }
    perf_list.append(evaluator.eval(input_dict)[metric])

# --- Aggregate results ---
perf_metrics = float(np.mean(perf_list))  # mean MRR over all test edges
print(f"TGBL-WIKI MRR Test: {perf_metrics}")


  0%|          | 0/23621 [00:00<?, ?it/s]/root/venv/lib/python3.11/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
100%|██████████| 23621/23621 [06:18<00:00, 62.39it/s]TGBL-COIN MRR Test: 0.46322616934776306





# Run continuous-time temporal network models on your own dataset

TGB implementations are mainly based on [DyGLib](https://github.com/yule-BUAA/DyGLib), which easily support the addition of new datasets https://github.com/yule-BUAA/DyGLib/blob/master/DG_data/DATASETS_README.md

Concerning the evaluation setting, TGB adopts the inductive negative sampling strategy available on DyGLib.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=44ea447d-a946-4e9f-93c8-1e95904db5b4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>